In [ ]:
import os
import glob
from PIL import Image
import numpy as np
import shutil
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

# CSV 파일 읽기
df = pd.read_csv("/content/grid_centers (1).csv")

# 'index' 열 이름을 'image_name'으로 변경
df.rename(columns={'파일이름': 'image_name'}, inplace=True)
df = df.iloc[:-1]

# (x, y) 형태의 문자열을 float 값으로 변환하는 함수
def split_coord(coord):
    # 괄호 제거 후 쉼표로 분리
    coord = coord.strip("()")
    x_str, y_str = coord.split(",")
    return float(x_str.strip()), float(y_str.strip())

# "중심좌표" 열의 값을 분리하여 두 개의 새 열 생성
df[['center_x_5186', 'center_y_5186']] = df['중심좌표'].apply(lambda x: pd.Series(split_coord(x)))

# 기존 "중심좌표" 열 삭제
df.drop(columns=['중심좌표'], inplace=True)

# 결과를 CSV 파일로 저장
df.to_csv("gridcenter.csv", index=False)


In [ ]:
import cv2
import numpy as np
import pandas as pd
# 입력 폴더와 출력 폴더 설정
input_folder = "/content/mask_pr/"  # 원본 이미지 폴더
output_folder = "/content/segmented_images"  # ID가 표시된 세그멘테이션 이미지 저장 폴더
csv_folder = "/content/segmentation_csv2"  # 빌딩 ID 및 면적 정보를 저장할 CSV 폴더

# 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

# PNG 파일만 선택
image_files = [f for f in os.listdir(input_folder) if f.endswith(".png")]

for image_file in image_files:
    # 이미지 읽기 (바이너리 마스크)
    image_path = os.path.join(input_folder, image_file)
    binary_mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Connected Components 분석
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_mask, connectivity=4)

    # CSV 저장을 위한 리스트 (벡터 연산)
    building_data = np.column_stack((np.arange(1, num_labels),  # Building_ID (배경 제외)
                                     stats[1:, cv2.CC_STAT_AREA],  # 면적
                                     centroids[1:, 0].astype(int),  # 중심 X
                                     centroids[1:, 1].astype(int)))  # 중심 Y

    # CSV 파일 저장
    csv_filename = os.path.join(csv_folder, image_file.replace(".png", ".csv"))
    pd.DataFrame(building_data, columns=["Building_ID", "Area", "Center_X", "Center_Y"]).to_csv(csv_filename, index=False)

    # ID가 표시된 이미지 생성
    segmented_image = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
    for label, center_x, center_y in zip(building_data[:, 0], building_data[:, 2], building_data[:, 3]):
        cv2.putText(segmented_image, str(int(label)), (center_x, center_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # 세그멘테이션 이미지 저장
    output_image_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_image_path, segmented_image)

print("Processing complete. Segmented images and CSV files are saved.")


Processing complete. Segmented images and CSV files are saved.


In [ ]:
# prompt: count the whole csv files data in the folder

import pandas as pd
import glob

# CSV 파일들이 있는 디렉토리 경로
csv_directory = "/content/segmentation_csv2"

# 디렉토리 내의 모든 CSV 파일 목록 가져오기
csv_files = glob.glob(os.path.join(csv_directory, "*.csv"))

total_rows = 0

# 각 CSV 파일의 행 수 계산
for file in csv_files:
    try:
        df = pd.read_csv(file)
        total_rows += len(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file} is empty.")
    except Exception as e:
        print(f"Error reading {file}: {e}")


print(f"Total number of rows across all CSV files: {total_rows}")


Total number of rows across all CSV files: 151461


In [ ]:
import cv2
import math
import pandas as pd
import numpy as np
import gc
import os

def calculate_angle(x1, y1, x2, y2, x3, y3, x4, y4):
    v1 = (x2 - x1, y2 - y1)
    v2 = (x4 - x3, y4 - y3)
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    mag_v1 = math.hypot(v1[0], v1[1])
    mag_v2 = math.hypot(v2[0], v2[1])
    cos_theta = dot_product / (mag_v1 * mag_v2)
    cos_theta = max(-1.0, min(1.0, cos_theta))
    angle = math.degrees(math.acos(cos_theta))
    return 180 - angle if angle > 90 else angle

def inverse_haversine(lat, lon, distance, angle):
    R = 6371000
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    angle_rad = math.radians(angle)

    new_lat = math.asin(math.sin(lat_rad)*math.cos(distance/R) +
                       math.cos(lat_rad)*math.sin(distance/R)*math.cos(angle_rad))
    new_lon = lon_rad + math.atan2(math.sin(angle_rad)*math.sin(distance/R)*math.cos(lat_rad),
                                  math.cos(distance/R)-math.sin(lat_rad)*math.sin(new_lat))
    return math.degrees(new_lat), math.degrees(new_lon)

# CSV 입력 및 경로 설정
input_df = pd.read_csv('/content/gridcenter.csv')
results = []
batch_size = 3
base_path = "/content/mask_pr"

# 배치 처리 루프
for batch_idx in range(0, len(input_df), batch_size):
    batch_df = input_df.iloc[batch_idx:batch_idx+batch_size]
    batch_results = []

    for idx, row in batch_df.iterrows():
        img_name = str(int(row['image_name']))
        img_path = os.path.join(base_path, f"{img_name}.png")

        try:
            # 파일 존재 여부 확인
            if not os.path.exists(img_path):
                print(f"파일 누락: {img_path}")
                continue

            center_lat = float(row['center_x_5186'])
            center_lon = float(row['center_y_5186'])

            # 그리드 계산
            lat_radian = math.radians(center_lat)
            grid_real = 0.51 * 1024
            lat_dis = 111320
            lon_dis = lat_dis * math.cos(lat_radian)
            grid_lat = grid_real / lat_dis
            grid_lon = grid_real / lon_dis
            SW_lat = center_lat - (grid_lat / 2)
            SW_lon = center_lon - (grid_lon / 2)

            # 이미지 로드
            mask = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"이미지 로드 실패: {img_path}")
                continue

            # 객체 분석
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, 4)

            # 개별 객체 처리
            for obj_id in range(1, num_labels):
                x, y = centroids[obj_id]

                # 좌하단 거리 계산
                dis = math.hypot(x, 1024-y)  # Y축 반전
                real_dis = dis * 0.51

                # 방위각 계산
                angle = calculate_angle(0, 1024, 1024, 1024, x, y, 0, 1024)
                bearing = (90 - angle) % 360

                # 좌표 변환
                obj_lat, obj_lon = inverse_haversine(SW_lat, SW_lon, real_dis, bearing)


                batch_results.append([
                    row['image_name'],
                    obj_id,
                    obj_lat,
                    obj_lon,
                    stats[obj_id, cv2.CC_STAT_AREA]
                ])



            del mask, labels, stats, centroids

        except Exception as e:
            print(f"{img_path} 처리 중 오류: {str(e)}")
            continue

    # 배치 결과 저장
    results.extend(batch_results)
    del batch_df, batch_results
    gc.collect()

pd.DataFrame(
    results,
    columns=['image_name','id','lat','lon','pixels']
).to_csv('buildings.csv', index=False)


print("처리 완료! buildings.csv 확인")

파일 누락: /content/mask_pr/459.png
파일 누락: /content/mask_pr/460.png
파일 누락: /content/mask_pr/540.png
파일 누락: /content/mask_pr/541.png
파일 누락: /content/mask_pr/542.png
파일 누락: /content/mask_pr/550.png
파일 누락: /content/mask_pr/551.png
파일 누락: /content/mask_pr/622.png
파일 누락: /content/mask_pr/623.png
파일 누락: /content/mask_pr/624.png
파일 누락: /content/mask_pr/625.png
파일 누락: /content/mask_pr/626.png
파일 누락: /content/mask_pr/627.png
파일 누락: /content/mask_pr/630.png
파일 누락: /content/mask_pr/631.png
파일 누락: /content/mask_pr/632.png
파일 누락: /content/mask_pr/633.png
파일 누락: /content/mask_pr/634.png
파일 누락: /content/mask_pr/635.png
파일 누락: /content/mask_pr/703.png
파일 누락: /content/mask_pr/704.png
파일 누락: /content/mask_pr/705.png
파일 누락: /content/mask_pr/706.png
파일 누락: /content/mask_pr/707.png
파일 누락: /content/mask_pr/708.png
파일 누락: /content/mask_pr/709.png
파일 누락: /content/mask_pr/710.png
파일 누락: /content/mask_pr/711.png
파일 누락: /content/mask_pr/712.png
파일 누락: /content/mask_pr/713.png
파일 누락: /content/mask_pr/714.png
파일 누락: /

In [ ]:
import numpy as np

# 원본 격자 생성 (68 x 82)
original_grid = np.arange(1, 68 * 82 + 1).reshape(68, 82)

# 그리드 크기 정의
grid_size = 12

# 결과를 저장할 리스트
grids = []

# 상단 그리드
for j in range(0, 82, grid_size):
    grid = original_grid[:grid_size, j:j+grid_size].flatten().tolist()
    grids.append(grid)

# 하단 그리드
for j in range(0, 82, grid_size):
    grid = original_grid[-grid_size:, j:j+grid_size].flatten().tolist()
    grids.append(grid)

# 좌측 그리드 (상단과 하단 제외)
for i in range(grid_size, 68 - grid_size, grid_size):
    grid = original_grid[i:i+grid_size, :grid_size].flatten().tolist()
    grids.append(grid)

# 우측 그리드 (상단과 하단 제외)
for i in range(grid_size, 68 - grid_size, grid_size):
    grid = original_grid[i:i+grid_size, -grid_size:].flatten().tolist()
    grids.append(grid)

# 결과 확인
for i, grid in enumerate(grids):
    print(f"그리드{i+1}: {grid}")

print(f"총 그리드 수: {len(grids)}")


그리드1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914]
그리드2: [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 423, 424, 425, 4

In [ ]:
import pandas as pd
import os

def merge_csvs(csv_list):
    dfs = []
    for csv in csv_list:
        if os.path.exists(csv):
            df = pd.read_csv(csv)
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

merged_grids = {}

for i, grid in enumerate(grids, 1):
    csv_list = [f"{num}.csv" for num in grid]
    merged_df = merge_csvs(csv_list)
    merged_grids[f"grid_{i}"] = merged_df

# 결과 확인
for grid_name, merged_df in merged_grids.items():
    print(f"{grid_name}: {len(merged_df)} rows")

print(f"\n총 그리드 수: {len(merged_grids)}")

In [ ]:
import numpy as np
import cv2
import os

# 기본 설정
grid_size = 12  # 개별 그리드 크기 (12x12)
image_size = 1024  # 개별 이미지 크기
grid_output_size = (image_size * grid_size, image_size * grid_size)  # 12x12 크기 이미지 (12288x12288)

# 전체 도넛 모양의 이미지 크기 (69632 × 83968)
full_output_size = (image_size * 68, image_size * 82)

# 전체 이미지 (초기: 빨간색)
full_image = np.full((*full_output_size, 3), (255, 0, 0), dtype=np.uint8)

# 도넛 모양의 그리드 배치
donut_grids = {
    "top": [(0, j) for j in range(0, 82, grid_size)],  # 상단
    "bottom": [(68-grid_size, j) for j in range(0, 82, grid_size)],  # 하단
    "left": [(i, 0) for i in range(grid_size, 68-grid_size, grid_size)],  # 좌측
    "right": [(i, 82-grid_size) for i in range(grid_size, 68-grid_size, grid_size)]  # 우측
}

# 이미지 폴더 경로 설정
image_dir = "images"

# 그리드 개별 이미지 생성
grid_count = 1
for section, positions in donut_grids.items():
    for (i, j) in positions:
        grid_image = np.full((*grid_output_size, 3), (0, 0, 255), dtype=np.uint8)  # 파란색 채우기

        for row in range(grid_size):
            for col in range(grid_size):
                img_num = (i + row) * 82 + (j + col) + 1  # 이미지 파일명 (예: 1.png)
                img_path = os.path.join(image_dir, f"{img_num}.png")

                row_pos = row * image_size
                col_pos = col * image_size

                if os.path.exists(img_path):  # 이미지가 존재하면 배치
                    img = cv2.imread(img_path)
                else:  # 없으면 파란색
                    img = np.full((image_size, image_size, 3), (0, 0, 255), dtype=np.uint8)

                grid_image[row_pos:row_pos+image_size, col_pos:col_pos+image_size] = img

        # 저장
        output_path = f"grid_{grid_count}.png"
        cv2.imwrite(output_path, grid_image)
        print(f"✅ 저장 완료: {output_path}")

        # 전체 이미지에 배치
        # full_row_start = i * image_size
        # full_col_start = j * image_size
        # full_image[full_row_start:full_row_start + grid_output_size[0],
        #            full_col_start:full_col_start + grid_output_size[1]] = grid_image

        grid_count += 1

# 전체 도넛 이미지 저장
# full_output_path = "full_grid.png"
# cv2.imwrite(full_output_path, full_image)
# print(f"✅ 전체 도넛 그리드 이미지 저장 완료: {full_output_path}")

print(f"총 {grid_count - 1}개의 개별 그리드 이미지가 생성되었습니다.")


ValueError: could not broadcast input array from shape (12288,12288,3) into shape (12288,10240,3)